# Uso de Resultados Individuales

A continuación ejemplificamos el modo de consumo de los servicios del API de Desempeño que nos permitirá realizar la consulta de resultados indivuales por colaborador

In [37]:
import requests
import json
import pandas as pd

In [ ]:
performance_api_url = "https://performance-reports-api.rankmi.com"

## Autenticación

Realizamos la autenticación con token de usuario previamente extraido de la configuración de Falabella

In [9]:
secret_key = "AQUI-SECRET-TOKEN"

In [33]:
query = """mutation SignIn {
  signIn(input:{
    secretKey:"%s",
    clientMutationId:""
  }) {
    jwt
    errors
    code
  }
}""" % secret_key

In [ ]:
url = "{}/authentication/sign_in".format(performance_api_url)

In [11]:
r = requests.post(url, json={'query':query})

if r.status_code != 200:
    raise ValueError("Error inesperado de Autenticacion: {}".format(r.text))

json_data = json.loads(r.text)
jwt = json_data["data"]["signIn"]["jwt"] if 'data' in json_data and 'signIn' in json_data['data'] else None
    
if jwt is None:
    raise ValueError("Error de Autenticacion: {}".format(json_data["data"]["signIn"]["errors"]))

headers = { "Authorization" : jwt }

## Procesos

De la siguiente forma podremos consultar todos los procesos disponibles de la empresa, en el ejemplo buscaremos uno en especifico que luego utilizaremos para la consulta de resultados individuales

In [34]:
process = None
process_id = "AQUI-ID-PROCESO"

In [35]:
query_process = """query {
     enterpriseProcesses {
         id
         token
         name
         description
         status
         startDate
         dueDate
   }
}"""

In [ ]:
url_process = "{}/enterprise-processes".format(performance_api_url)

In [27]:
rprocess = requests.post(url_process, json={'query':query_process}, headers=headers)

if rprocess.status_code != 200:
    raise ValueError("Error inesperado de Consulta de Procesos:", rprocess.text)

json_process = json.loads(rprocess.text)

enterpriseProcesses = json_process['data']['enterpriseProcesses'] if 'data' in json_process and 'enterpriseProcesses' in json_process['data'] else None

if enterpriseProcesses is None:
    raise ValueError("Error de Consulta Procesos: {}".format(json_collab['errors'][0]['message']))
    
process, process_name = None, None
for p in enterpriseProcesses:
    if p["id"] == process_id:
        process = p["token"]
        process_name = p["name"]
        
if process is None:
    raise ValueError("Error Proceso No Encontrado: {}".format(process_id))

## Colaboradores del Proceso

Aqui consultaremos a los colaboradores que estan incluidos proceso de desempeño elegido previamente

In [28]:
query_collaborators = """query {
  collaboratorsList(processToken: "%s"
  page: 3
  lang: "en"){
     id
     rut
     name
     token
     email
  }
}""" % process

In [ ]:
url_collab = "{}/collaborators/index".format(performance_api_url)

In [18]:
rcollab = requests.post(url_collab, json={'query':query_collaborators}, headers=headers)

if rcollab.status_code != 200:
    raise ValueError("Error inesperado de Consulta Colaboradores:", rcollab.text)

json_collab = json.loads(rcollab.text)

collaboratorsList = json_collab['data']['collaboratorsList'] if 'data' in json_collab and 'collaboratorsList' in json_collab['data'] else None

if collaboratorsList is None:
    raise ValueError("Error de Consulta Colaboradores: {}".format(json_collab['errors'][0]['message']))


## Resultados Individuales de colaboradores

Ahora con el listado de colaboradores, iteraremos para extraer toda la información de resultados individuales. El siguiente es un método que ejemplifica su uso:

In [ ]:
url_individual_res = "{}/individual-results".format(performance_api_url)

def consulta_individual_colaborador(proceso, colaborador):
    query_ind_result = """query {
      individualResult(processToken: "%s"
      collaboratorToken: "%s"
      lang: "en"){
         id
         name
         token
         area
         position
         email
         evaluators
         generalAverage{
          evaluationDomain
          evaluationResult
        }
        domains{
          domainId
          name
          domainAverage
          competencesAverage
          goalsAverage
          competences
          categoriesConducts
        }
      }
    }""" % (proceso, colaborador)
    
    rindividual = requests.post(url_individual_res, json={'query':query_ind_result}, headers=headers)
    
    if rindividual.status_code != 200:
        raise ValueError("Error inesperado de Consulta Colaboradores:", rindividual.text)

    json_individ = json.loads(rindividual.text)

    individualResult = json_individ['data']['individualResult'] if 'data' in json_individ and 'individualResult' in json_individ['data'] else None

    if individualResult is None:
        raise ValueError("Error de Consulta Individual: {}".format(json_individ['errors'][0]['message']))
        
    return individualResult  

En este momento estará habilitado para realizar la consulta individual de un colaborador en específico o iterar todos los colaboradores para hacer una consulta completa y general de sus resultados individuales

In [ ]:
consulta_individual_colaborador(process, "AQUI-TOKEN-COLABORADOR")